Load required componets and earthengine

**this was tested at the root dir of se.plan**

In [ ]:
from component import model
from component import scripts
import ee
ee.Initialize()

Example loading a previous recipe to run se.plan without ui

In [2]:

import json
path = '/home/jdilger/module_results/se.plan/SLV/recipe_2022-05-23.json'
with open(path, 'r') as f:
  data = json.load(f)

data.keys()

dict_keys(['aoi_model', 'layer_model', 'question_model'])

In [17]:
# get priorities and constraints AS STRINGS
# wlc function expects this
priorities = data['question_model']['priorities']
constraints = data['question_model']['constraints']
print('wlc priorities input type should be a string :',type(priorities))
print('wlc constraints input type should be a string :',type(constraints))

wlc priorities input type should be a string <class 'str'>
wlc constraints input type should be a string <class 'str'>


In [5]:
# todo, code simple way to pull aoi from recipe. using gee for now
table = ee.FeatureCollection("FAO/GAUL/2015/level0")
geometry = ee.Geometry.Point([-89.1355, 13.6937])
aoi = table.filterBounds(geometry)


Use the se.plan base model to load the default layer list
or
use layerlist from recipe (this is the best to use if custom layers have been added).


In [25]:
base_layers_model = model.CustomizeLayerModel()
layer_list = base_layers_model.layer_list
type(layer_list)
type(data['layer_model']['layer_list'])

list

In [13]:
# wlc returns wlc_out, benefit_list, constraint_list, cost_list
wlc = scripts.wlc(layer_list, constraints, priorities, aoi)
output = wlc[0]

In [8]:
import geemap
Map = geemap.Map()

In [9]:
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [16]:
Map.addLayer(output,{'min':0,'max':5,'palette':['black','red','blue','yellow','green','purple']},'wlc output')

### get dashboard stats

In [20]:
a = scripts.get_summary_statistics(wlc, 'test-elsav', aoi, layer_list)
json.loads(a)

{'benefit': [{'woodfuel_harvest': {'total': [6.2153071598810605],
    'values': [6.160226015852092]}},
  {'plantation_growth_rates': {'total': [6.703870658466306],
    'values': [6.614907734702729]}},
  {'forest_job': {'total': [0.014178218332882336],
    'values': [0.013925709551751767]}},
  {'ground_carbon': {'total': [2.67318169360335],
    'values': [2.6712878946802108]}},
  {'biodiversity_intactness': {'total': [0.8837665322113594],
    'values': [0.8785625239145877]}},
  {'endangered_species': {'total': [4.917549363470223],
    'values': [4.772135457736469]}}],
 'constraint': [{'city_access': {'total': [2073183.399678905],
    'values': [0.0]}},
  {'ecozones': {'total': [2073183.399678905], 'values': [0.0]}},
  {'land_cover': {'total': [2073183.399678905], 'values': [0.0]}},
  {'protected_areas': {'total': [2073183.399678905], 'values': [0.0]}},
  {'slope': {'total': [2073183.399678905], 'values': [0.0]}},
  {'elevation': {'total': [2073183.399678905], 'values': [0.0]}},
  {'natu